# BASICS 02 - HOTS in PyTorch

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ../hots
import tonic, torch, os, pickle
from tqdm import tqdm
from network import network
from layer import mlrlayer
from timesurface import timesurface
from utils import get_loader, make_histogram_classification, HOTS_Dataset

print(f'Tonic version installed -> {tonic.__version__}')

print(f'Number of GPU devices available: {torch.cuda.device_count()}')
for N_gpu in range(torch.cuda.device_count()):
    print(f'GPU {N_gpu+1} named {torch.cuda.get_device_name(N_gpu)}')

/home/antoine/homhots/hotsline/hots
Tonic version installed -> 1.0.15
Number of GPU devices available: 1
GPU 1 named GeForce RTX 2080 Ti


## Loading of the dataset for the clustering phase

In [3]:
transform = tonic.transforms.NumpyAsType(int)
trainset = tonic.datasets.POKERDVS(save_to='../../Data/', train=True, transform=transform)
testset = tonic.datasets.POKERDVS(save_to='../../Data/', train=False, transform=transform)
loader = get_loader(trainset)
num_sample_train = len(loader)
num_sample_test = len(testset)
print(f'number of samples in the training set: {len(loader)}')

number of samples in the training set: 48


## Initialization of the network

In [4]:
name = 'homeohots'
homeo = True
timestr = '2022-04-22'
dataset_name = 'poker'

Rz = [2, 4]
N_neuronz = [4, 8]
tauz = [1e3, 2e3]

hots = network(name, dataset_name, timestr, trainset.sensor_size, nb_neurons = N_neuronz, tau = tauz, R = Rz, homeo = homeo)

## Unsupervised clustering

In [5]:
if not os.path.exists('../Records/'):
    os.mkdir('../Records/')
    os.mkdir('../Records/networks/')
filtering_threshold = [2*Rz[L] for L in range(len(Rz))]
path = '../Records/networks/'+hots.name+'.pkl'
if not os.path.exists(path):
    hots.clustering(loader, trainset.ordering, filtering_threshold, record = True)

## Training of the classification layer

In [6]:
jitter = (None, None)

train_path = f'../Records/output/train/{hots.name}_{num_sample_train}_{jitter}/'
test_path = f'../Records/output/test/{hots.name}_{num_sample_test}_{jitter}/'

tau_cla = 1e4

transform = tonic.transforms.Compose([tonic.transforms.ToTimesurface(sensor_size=trainset.sensor_size, tau=tau_cla, decay="exp")])

transform = tonic.transforms.NumpyAsType(int)
testset_output = HOTS_Dataset(test_path, trainset.sensor_size, dtype=trainset.dtype, transform=transform)
trainset_output = HOTS_Dataset(train_path, trainset.sensor_size, dtype=trainset.dtype, transform=transform)

In [15]:
def fit_mlr(loader, 
            model_path,
            tau_cla,
            learning_rate,
            betas,
            num_epochs,
            ts_size,
            ordering,
            n_classes,
            num_workers=0):
    
    if os.path.exists(model_path):
        with open(model_path, 'rb') as file:
            classif_layer, losses = pickle.load(file)
    
    else:
        torch.set_default_tensor_type("torch.DoubleTensor")
        criterion = torch.nn.BCELoss(reduction="mean")
        amsgrad = True #or False gives similar results
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        N = ts_size[0]*ts_size[1]*ts_size[2]

        classif_layer = mlrlayer(N, n_classes, device=device)
        classif_layer.train()
        optimizer = torch.optim.Adam(
            classif_layer.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
        )

        for epoch in tqdm(range(int(num_epochs))):
            losses = []
            for events, label in loader:
                X, ind_filtered = timesurface(events.squeeze(0).squeeze(0), (ts_size[0], ts_size[1], ts_size[2]), ordering, tau = tau_cla, device=device)
                
                X, label = X.to(device) ,label.to(device)
                X = X.reshape(X.shape[0], N)

                outputs = classif_layer(X)

                n_events = X.shape[0]
                labels = label*torch.ones(n_events).type(torch.LongTensor).to(device)
                labels = torch.nn.functional.one_hot(labels, num_classes=n_classes).type(torch.DoubleTensor).to(device)

                loss = criterion(outputs, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())

        with open(model_path, 'wb') as file:
            pickle.dump([classif_layer, losses], file, pickle.HIGHEST_PROTOCOL)

        return classif_layer, losses

In [ ]:
loader = get_loader(trainset_output)
model_path = 'test.pkl'

num_workers = 0
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 3 #2 ** 5 + 1
ts_size = (trainset.sensor_size[0],trainset.sensor_size[1],N_neuronz[-1])

classif_layer, losses = fit_mlr(loader, model_path, tau_cla, learning_rate, betas, num_epochs, ts_size, trainset.ordering, len(trainset.classes))

  0%|                                                                                                                                                                                                                                                   | 0/3 [00:00<?, ?it/s]